In [1]:
"""Solution of ray tutorial exercise 08
Speed up Serialization

GOAL: The goal of this exercise is to illustrate how to speed up serialization by using ray.put.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import pickle

ray.init(redirect_output=True, num_cpus=4)

neural_net_weights = {'variable{}'.format(i): np.random.normal(size=1_000_000)
  for i in range(50)}

Waiting for redis server at 127.0.0.1:52697 to respond...
Waiting for redis server at 127.0.0.1:19019 to respond...
Starting local scheduler with the following resources: {'CPU': 4, 'GPU': 2}.

View the web UI at http://localhost:8890/notebooks/ray_ui77209.ipynb?token=7b52c1cdcadb55a3197aeb69ccab301a555a1f2ac1730678



"\nprint('Ray - serializing')\n%time x_id = ray.put(neural_net_weights)\nprint('\nRay - deserializing')\n%time x_val = ray.get(x_id)\n\nprint('\npickle - serializing')\n%time serialized = pickle.dumps(neural_net_weights)\nprint('\npickle - deserializing')\n%time deserialized = pickle.loads(serialized)\n"

In [3]:
@ray.remote
def use_weights(weights, i):
    # ray.get(weights)
    return i

In [4]:
import time

In [5]:
# Sleep a little to improve the accuracy of the timing measurements below.
time.sleep(2.0)
start_time = time.time()

x_id = ray.put(neural_net_weights)
results = ray.get([use_weights.remote(x_id, i)
                   for i in range(20)])

end_time = time.time()
duration = end_time - start_time

In [6]:
assert results == list(range(20))
assert duration < 1, ('The experiments ran in {} seconds. This is too '
                      'slow.'.format(duration))

print('Success! The example took {} seconds.'.format(duration))

Success! The example took 0.1627817153930664 seconds.


In [7]:
print('Ray - serializing')
%time x_id = ray.put(neural_net_weights)
print('\nRay - deserializing')
%time x_val = ray.get(x_id)

print('\npickle - serializing')
%time serialized = pickle.dumps(neural_net_weights)
print('\npickle - deserializing')
%time deserialized = pickle.loads(serialized)

Ray - serializing
CPU times: user 0 ns, sys: 272 ms, total: 272 ms
Wall time: 149 ms

Ray - deserializing
CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 760 µs

pickle - serializing
CPU times: user 124 ms, sys: 116 ms, total: 240 ms
Wall time: 240 ms

pickle - deserializing
CPU times: user 60 ms, sys: 56 ms, total: 116 ms
Wall time: 113 ms
